<a href="https://colab.research.google.com/github/kvolle/4777_project/blob/master/Speed_Challenge_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Comma AI Speed Challenge**

  This notebook will contain (hopefully) all of the functions you need to import the data into your model.

  ***Be sure to train with GPU acceleration enabled***

**Import Statements**

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

**Custom Data Generator**

This works (I think) for giving two sequential images to a Keras Functional model as well as the velocity associated with the second image.

At this point, don't worry about how this works. If you need something changed or fixed, just ask. This is the boring part anyways.

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.direct = "./drive/My Drive/commai_dataset/"

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Find list of IDs
        list_IDs_temp = self.list_IDs[index]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(self.direct+"data/data_" + list_IDs_temp +".npy")
        x1 = X[0:101,:,:,:]
        x2 = X[1:102,:,:,:]
        y = np.load(self.direct+"labels/label_" + list_IDs_temp +".npy")
        y = y[1:]

        return [x1, x2], y

**Define custom loss function**

This is not well tested, neither is it optimized. You might not even want to use this function.

Keras backend functions are a powerful tool for writing custom loss functions. To define a loss function it just has to accept *y_true* and *y_pred* as arguments and return a float.

To use your new loss function, change the argument in *model.compile()*.

In [0]:
def sum_sq_err(y_true, y_pred):
    return tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred))

**Define the test-train split and create the Data Generator**

In [0]:
params = {'dim': (110,320),
          'batch_size': 101,
          'n_classes': 1,
          'n_channels': 3,
          'shuffle': False}

train_data = []
train_label = []
valid_data = []
valid_label = []

for i in range(70):
    train_data.append("%03d" %i)
    train_data.append("%03d" %(i+100))

for i in range(70, 100):
    valid_data.append("%03d" %i)
    valid_data.append("%03d" %(i+100))

partition={'train':train_data, 'validation':valid_data}
labels = {'train': train_label,'validation':valid_label}

training_generator = DataGenerator(partition['train'], labels['train'], **params)
validation_generator = DataGenerator(partition['train'], labels['train'], **params)

**Define the input layers**

In [0]:
input_A = tf.keras.layers.Input(shape=(110, 320, 3), name="first_image")
input_B = tf.keras.layers.Input(shape=(110, 320, 3), name="second_image")

**Define the model**

This model is a particularly ridiculous placeholder. Make yours better

In [0]:
flat1 = tf.keras.layers.Flatten()(input_A)
flat2 = tf.keras.layers.Flatten()(input_B)
out = tf.keras.layers.Dot(axes=1)([flat1, flat2])

model = tf.keras.models.Model(inputs=[input_A, input_B], outputs=[out])

**How ridiculous? Run this next cell and find out.**

(Note how many parameters there are, at least it trains fast)

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_image (InputLayer)        [(None, 110, 320, 3) 0                                            
__________________________________________________________________________________________________
second_image (InputLayer)       [(None, 110, 320, 3) 0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 105600)       0           first_image[0][0]                
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 105600)       0           second_image[0][0]               
____________________________________________________________________________________________

**Declare the optimizer and loss function, then compile your *less ridiculous*  model**

In [0]:
_optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=_optimizer, loss = sum_sq_err)

**Train using the fit_generator**

In [0]:
model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5)

Epoch 1/5
1/1 [==============================] - 1s 725ms/step - loss: 103425191830842507264.0000 - val_loss: 103425191830842507264.0000
Epoch 2/5
1/1 [==============================] - 0s 371ms/step - loss: 103425191830842507264.0000 - val_loss: 103425191830842507264.0000
Epoch 3/5
1/1 [==============================] - 0s 385ms/step - loss: 103425191830842507264.0000 - val_loss: 103425191830842507264.0000
Epoch 4/5
1/1 [==============================] - 0s 373ms/step - loss: 103425191830842507264.0000 - val_loss: 103425191830842507264.0000
Epoch 5/5
1/1 [==============================] - 0s 378ms/step - loss: 103425191830842507264.0000 - val_loss: 103425191830842507264.0000


In [0]:
!ls

drive  sample_data
